In [1]:
#This notebook fine-tunes Llama-3.1-8B-Instruct to explain medical diagnoses to patients using the Med-EASi dataset.

!pip install -q "transformers>=4.43.0" peft bitsandbytes accelerate datasets evaluate wandb huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
from getpass import getpass

HF_TOKEN = getpass("Enter your Hugging Face token (with access to Llama-3.1-8B): ")
login(HF_TOKEN)


Enter your Hugging Face token (with access to Llama-3.1-8B): ··········


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
ds = load_dataset("cbasu/Med-EASi")
print(ds)
print("Train sample:\n", ds["train"][0])

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1397 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/196 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 1397
    })
    validation: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 196
    })
    test: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 300
    })
})
Train sample:
 {'Expert': '75-90 % of the affected people have mild intellectual disability.', 'Simple': "People with syndromic intellectual disabi

In [ ]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
    token=HF_TOKEN,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    token=HF_TOKEN,
)

model.config.use_cache = False


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605


In [ ]:
MAX_LENGTH = 256 #change later if needed

SYSTEM_PROMPT = (
    "You are a medical explainer. Your job is to explain medical diagnoses "
    "in clear, patient-friendly English. Use short sentences, avoid medical "
    "jargon when possible, and do not add information that is not in the "
    "original text.\n\n"
)

def build_training_prompt(expert_text: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": (
                "Explain the following medical diagnosis to a patient in simple, "
                "everyday language:\n\n"
                f"{expert_text}\n\n"
                "Patient-friendly explanation:"
            ),
        },
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return prompt

def preprocess_function(batch):
    experts = batch["Expert"]
    simples = batch["Simple"]

    prompts = [build_training_prompt(e) for e in experts]
    targets = simples

    full_texts = [p + t + tokenizer.eos_token for p, t in zip(prompts, targets)]

    tokenized = tokenizer(
        full_texts,
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length",
    )

    input_ids = tokenized["input_ids"]
    labels = []
    attention_masks = tokenized["attention_mask"]

    for i, full_ids in enumerate(input_ids):
        #tokenize prompt to get number of prompt tokens
        prompt_ids = tokenizer(
            prompts[i],
            add_special_tokens=False,
            max_length=MAX_LENGTH,
            truncation=True,
        )["input_ids"]

        label_ids = full_ids.copy()
        prompt_len = min(len(prompt_ids), len(label_ids))

        #we only want loss on answer tokens, so mask out the prompt tokens
        label_ids[:prompt_len] = [-100] * prompt_len
        labels.append(label_ids)

    tokenized["labels"] = labels
    return tokenized

cols_to_remove = ds["train"].column_names

tokenized_ds = ds.map(
    preprocess_function,
    batched=True,
    remove_columns=cols_to_remove,
)

tokenized_ds


Map:   0%|          | 0/1397 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1397
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 196
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [ ]:
def data_collator(features):
    batch = {}
    for key in ["input_ids", "attention_mask", "labels"]:
        batch[key] = torch.stack(
            [torch.tensor(f[key]) for f in features]
        )
    return batch

In [ ]:
wandb.init(
    project="med-easi-llama3",
    name="llama3_1_8b_model",
)

training_args = TrainingArguments(
    output_dir="checkpoints/llama3_1_8b_med_easi",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    learning_rate=2e-4,
    warmup_ratio=0.05,
    logging_steps=20,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to=["wandb"],
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

train_result = trainer.train()
print("Training finished.")

/tmp/ipython-input-1526134375.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.226400,0.376193
2,0.133900,0.403679


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Training finished.


In [ ]:
HF_TOKEN = getpass("Enter token with write access to upload model to HF repo")
login(HF_TOKEN)

In [ ]:
repo_id = "smedara/llama3-med-easi-explainer"
model.push_to_hub(repo_id=repo_id, private=True)
tokenizer.push_to_hub(repo_id)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   1%|          |  555kB / 83.9MB            

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpxkz5ltdl/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

CommitInfo(commit_url='https://huggingface.co/smedara/llama3-med-easi-explainer/commit/f1180ff60d998d8e1d2235b9de5b8a503c38bd6b', commit_message='Upload tokenizer', commit_description='', oid='f1180ff60d998d8e1d2235b9de5b8a503c38bd6b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/smedara/llama3-med-easi-explainer', endpoint='https://huggingface.co', repo_type='model', repo_id='smedara/llama3-med-easi-explainer'), pr_revision=None, pr_num=None)

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig



repo_id = "smedara/llama3-med-easi-explainer"

test_dataset = ds["test"]


tokenizer = AutoTokenizer.from_pretrained(repo_id)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    repo_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.eval()
print("Model loaded successfully.")


def generate_predictions(batch):

    prompts = []
    for expert_text in batch["Expert"]:
        # Wrap the input text in the Llama 3 User/Assistant structure
        messages = [
            {"role": "user", "content": expert_text},
        ]
        # 'add_generation_prompt=True' adds the <|start_header_id|>assistant<|end_header_id|>\n prefix
        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        prompts.append(prompt)


    inputs = tokenizer(
        prompts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )


    inputs = {k: v.to(model.device) for k, v in inputs.items()}


    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=180,
            do_sample=True,
            top_k=70,
            top_p=0.99,
            temperature=0.8,
            eos_token_id=tokenizer.eos_token_id,

            pad_token_id=tokenizer.eos_token_id
        )


    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=False)

    predictions = []
    for output in decoded_outputs:

        try:
            # Look for the start of the assistant's content after the header
            response_start = output.split('<|end_header_id|>')[-1].strip()
            # Remove the end-of-turn token if it was generated
            final_prediction = response_start.split('<|eot_id|>')[0].strip()
        except:
            final_prediction = output # Fallback

        predictions.append(final_prediction)

    batch["pred"] = predictions
    return batch

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

Model loaded successfully.


In [12]:
predictions = test_dataset.map(generate_predictions, batched=True, batch_size=8)


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [13]:
!pip install evaluate
!pip install sacremoses sacrebleu
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8d761c572e1f9971b22bf0dd168f8643a7d4973f7571725c31a294f0c6582e85
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [14]:

import evaluate

sari_metric = evaluate.load("sari")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
sari_score = sari_metric.compute(
    sources=predictions["Expert"],
    predictions=predictions["pred"],
    references=[[ref] for ref in predictions["Simple"]]
)

bleu_score = bleu_metric.compute(
    predictions=predictions["pred"],
    references=[[ref] for ref in predictions["Simple"]]
)

rouge_score = rouge_metric.compute(
    predictions=predictions["pred"],
    references=predictions["Simple"]
)
print("SARI:", sari_score)
print("BLEU:", bleu_score)
print("ROUGE:", rouge_score)


SARI: {'sari': 30.576178794904774}
BLEU: {'bleu': 0.0013318113816146327, 'precisions': [0.4379615952732644, 0.22948328267477203, 0.1651017214397496, 0.13951612903225807], 'brevity_penalty': 0.0060715831426092475, 'length_ratio': 0.16382335148215366, 'translation_length': 1354, 'reference_length': 8265}
ROUGE: {'rouge1': np.float64(0.04708962149276673), 'rouge2': np.float64(0.026092052504372742), 'rougeL': np.float64(0.03988781850243422), 'rougeLsum': np.float64(0.03982655922485026)}


In [15]:
import pandas as pd

df = pd.DataFrame({
    "expert": predictions["Expert"],
    "reference_simple": predictions["Simple"],
    "model_simple": predictions["pred"]
})

df.to_csv("model_outputs.csv", index=False)
